In [ ]:
import pandas as pd
import folium
from collections import Counter
import numpy as np
import json

In [ ]:
#Read in CSV 
Drivers = pd.read_csv('Transportation_Network_Providers_-_Drivers.csv',sep=',')
Drivers.head()

In [ ]:
# Find drivers with more than 0 trips
Drivers_More_Than1 = Drivers[Drivers['NUMBER_OF_TRIPS']>0]

# Get subset of fields
Subset = Drivers_More_Than1[['ZIP','NUMBER_OF_TRIPS']]

# Remove all non zero values
Final = Subset[~Subset.applymap(np.isreal).all(1)]
Final_Zips = Final.groupby('ZIP').count()[0:-7].reindex()
Final_Zips = Final_Zips[~Final_Zips.index.str.contains("-")]

Final_Zips.index = pd.to_numeric(Final_Zips.index, errors='coerce')

# Subset data to only Illinois Zip Codes
Final_Zips = Final_Zips[Final_Zips.index >59999]
Final_Zips = Final_Zips[Final_Zips.index <70000]
Final_Zips.head()

In [ ]:
# Get list of zip codes
Zips = list(Final_Zips.index)

In [ ]:
# Pull in geojson
with open('il_illinois_zip_codes_geo.min.json','r') as jsonFile:
    data = json.load(jsonFile)
data

In [ ]:
# Loop through geojson to only return a subset of all features for the map
tmp = data
geozips = []
for i in range(len(tmp['features'])):
    if int(tmp['features'][i]['properties']['ZCTA5CE10']) in Zips:
        geozips.append(tmp['features'][i])
new_json = dict.fromkeys(['type','features'])
new_json['type'] = 'FeatureCollection'
new_json['features'] = geozips

In [ ]:
# Save new json
open('updated-geojson-il.json','w').write(
json.dumps(new_json,sort_keys=True,indent=4,separators=(',',': '))
)

In [ ]:
Final_Zips.reset_index(inplace=True)
Final_Zips['ZIP'] = Final_Zips['ZIP'].apply(str)
Final_Zips.head()

In [ ]:
m = folium.Map(location=[41.8781, -87.6298],width=500,height=700)
folium.Choropleth(
    geo_data=new_json,
    name='choropleth',
    data=Final_Zips,
    columns=['ZIP','NUMBER_OF_TRIPS'],
    key_on='feature.properties.ZCTA5CE10',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Number of Rideshare Drivers'
).add_to(m)

folium.LayerControl().add_to(m)

m

In [ ]:
m.save('index.html')